# Intro to Artificial Neural Networks with Keras

In Artificial Intelligence we get inspiration from nature. And it is in nature that sparked the idea of a artificual neural network. This is a way to mimic biological neurons in code. 

ANNs are at the core of Deep learning, they are versatile, poewrful and scalable, making them ideal to tackle large adn hihgly complex Machine Learning tasks. 

ANNs were first created in a paper in 1943 and it thought to be the next wave of AI. However it died down but now with some much data available and a ton of cheap computing power we are able to take advantage of this framework. 

The way to think of a artificial neuron, is that has 1 or more binary inputs and one binary output. The inputs are on and off switches. 

<img src='images/anns_logical_computation.png'/> </img>

* The first network on the left is the identiy function: if neuron A is activated, then neuron C gets activated as well but if neuron A is off then neuron C is off as well. 

* The second network performs a logical AND neuron C gets activated if either neuron A or neuron B is activated

* The third network performs a logical OR neuron C gets activated if either neuron A or neuron B is activated or both

* Finally, if we suppose that an input connection can inhibit the neurons's activity (which is the case with biological neurons), then the fourth networks computes a slighty more complex logical propsition: Neuron C is activated  only if Nueron A is actiave and neuron B is off. If neuron A isactivate all the time then you get a logical NOT neuron C is active when neuron B is off and vice versa. 

# Perceptron

This is one of the simplest ANN architectures, it is based on a neuron called the Threshold logic unit. The input and outputs are numbers instead of binary on and off switches. Each input connection is associated with a weight. The TLU computes a weighted sum of its input then applies a step funciton to that sum and outputs the results.

<img src="images/tlu.png"/></img>

The most common function is the Heaviside step function. 

